In [1]:
! pip install nltk 

  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.3 MB/s eta 0:00:00a 0:00:010m


In [69]:
from keras.layers import TextVectorization

2024-04-03 00:14:31.697392: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 00:14:32.228291: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
import nltk 
nltk.download("all") 

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [104]:
txt = "[CLS] I didn't A-list \x86 hit his [mask] I was at the store [SEP]" 
stop_words = set(stopwords.words('english'))

# txt = txt.replace("[CLS]", "CLS") 
# txt = txt.replace("]", "") 
# txt = txt.replace("[mask]", "MASK") 

tokenized = sent_tokenize(txt)


for i in tokenized: 
    wordsList = nltk.word_tokenize(i)
    
    # removing stop words from wordList
    # wordsList = [w for w in wordsList] 
    
    #  Using a Tagger. Which is part-of-speech 
    # tagger or POS-tagger. 
    tagged = nltk.pos_tag(wordsList)
    
    print(tagged)
    
#in cases of apostrophes, merge the word as a singular one 
real_tagged = [] 
count = 0 
while count < len(tagged): 
    if '\'' in tagged[count][0]: 
        real_tuple = (tagged[count-1][0] + tagged[count][0], tagged[count-1][1])
        # real_tagged[count-1][0] += tagged[count][0] 
        real_tagged[count-1] = real_tuple
    else:
        real_tagged.append(tagged[count])
        # print(f'real_tagged: {real_tagged}')
    count += 1
print(real_tagged) 

[('[', 'JJ'), ('CLS', 'NNP'), (']', 'NNP'), ('I', 'PRP'), ('did', 'VBD'), ("n't", 'RB'), ('A-list', 'NNP'), ('\x86', 'NNP'), ('hit', 'VBD'), ('his', 'PRP$'), ('[', 'JJ'), ('mask', 'NN'), (']', 'NN'), ('I', 'PRP'), ('was', 'VBD'), ('at', 'IN'), ('the', 'DT'), ('store', 'NN'), ('[', 'NNP'), ('SEP', 'NNP'), (']', 'NN')]
[('[', 'JJ'), ('CLS', 'NNP'), (']', 'NNP'), ('I', 'PRP'), ("didn't", 'VBD'), ('A-list', 'NNP'), ('\x86', 'NNP'), ('hit', 'VBD'), ('his', 'PRP$'), ('[', 'JJ'), ('mask', 'NN'), (']', 'NN'), ('I', 'PRP'), ('was', 'VBD'), ('at', 'IN'), ('the', 'DT'), ('store', 'NN'), ('[', 'NNP'), ('SEP', 'NNP'), (']', 'NN')]


In [19]:
list_txt = txt.split(" ")
print(list_txt)

['[CLS]', 'I', "didn't", 'A-list', 'hit', 'his', '[mask]', 'I', 'was', 'at', 'the', 'store', '[SEP]']


In [26]:
#create dictionary of {noun, adj, verb, adverb, prounoun} 
#map every part of speech to one of the above 
pos_vals = {"D", "J", "N", "P", "V", "R"}
map_to_pos = {"D":"verb", "J":"adj", "N":"noun", "P":"pronoun", "V":"verb", "R": "adverb"}

word_pos = [] 
indx = 0 
for word, pos in real_tagged: 
    if not(word == "[" or word == "mask" or word == "]" or word == "CLS" or word == "SEP" or pos[0] not in pos_vals):
        #for debugging
        # word_pos.append((word, map_to_pos[pos[0]])) #change this back to just a value of map_to_pos(pos[0]) after debugging
        word_pos.append(map_to_pos[pos[0]])
    else:
        if word == "]": 
            word_pos.pop(-1)
            word_pos.pop(-1)
        word_pos.append(None) 
    indx +=  1
print(word_pos)
    

[None, 'pronoun', 'verb', 'noun', 'verb', 'pronoun', None, 'pronoun', 'verb', None, 'verb', 'noun', None]


In [31]:
#allowed relationships 

relationships = {"noun": {"verb", "adj", "pronoun"},  "verb":{"noun", "adverb"}, "adj":{"noun", "pronoun"}, "pronoun":{"verb", "adj", "adverb", "noun"}, "adverb": {"verb"}}

In [6]:
import numpy as np 

In [33]:
mask = np.ones((len(list_txt), len(list_txt))) #including the [CLS] and [SEP] tags 
#relationships that we want noun->adj, adj->noun, verb->noun, noun->verb, adverb->verb, verb->adverb, pronouns->nouns, nouns->pronouns
mask_indx = 6 #adjusted for [CLS] and [SEP] tokens 

for row in range(mask.shape[0]):
    row_word_pos = word_pos[row] 
    if row_word_pos: #the [CLS], [SEP], and [MASK] tokens will be allowed to be influenced by every word 
        allowed_pos = relationships[row_word_pos] 
        for col in range(mask.shape[1]): 
            #for now, don't worry about the [mask] token not being allowed to influence the words 
            if word_pos[col] not in allowed_pos: 
                mask[row,col] = 0
            # else:
            #     mask[row, col] = 0 
np.fill_diagonal(mask, 1) #allow words to attend to themselves

print(txt)
print(mask)    

#the below is wrong 

[CLS] I didn't A-list hit his [mask] I was at the store [SEP]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [65]:
#currently this mask does not allow the [mask] token to attend to other words  
def generate_mask(txt, debug=False):
    txt = txt.replace("(", "")
    txt = txt.replace(")", "")
    # txt = re.sub(' +', ' ', txt)
    tokenized = sent_tokenize(txt)
    for i in tokenized: 
        wordsList = nltk.word_tokenize(i) 
        tagged = nltk.pos_tag(wordsList)
    real_tagged = [] 
    count = 0 
    while count < len(tagged): 
        if '\'' in tagged[count][0]: 
            real_tuple = (tagged[count-1][0] + tagged[count][0], tagged[count-1][1])
            # real_tagged[count-1][0] += tagged[count][0] 
            # print(f'length: {len(real_tagged)}')
            # print(f'count: {count}, {real_tagged}')
            real_tagged[-1] = real_tuple
        else:
            real_tagged.append(tagged[count])
            # print(f'real_tagged: {real_tagged}')
        count += 1
    if debug:
        print(f'Tags: {real_tagged}')
    list_txt = txt.split(" ")

    pos_vals = {"D", "J", "N", "P", "V", "R"}
    map_to_pos = {"D":"verb", "J":"adj", "N":"noun", "P":"pronoun", "V":"verb", "R": "adverb"}
    
    word_pos = [] 
    indx = 0 
    for word, pos in real_tagged: 
        if not(word == "[" or word == "mask" or word == "]" or word == "CLS" or word == "SEP" or pos[0] not in pos_vals):
            #for debugging
            # word_pos.append((word, map_to_pos[pos[0]])) #change this back to just a value of map_to_pos(pos[0]) after debugging
            word_pos.append(map_to_pos[pos[0]])
        else:
            if word == "]": 
                word_pos.pop(-1)
                word_pos.pop(-1)
            word_pos.append(None) 
        indx +=  1

    print(f'length of word pos (After removing extra spaces): {len(word_pos)}') 
    print(f'length of list_txt: {len(list_txt)}')
    relationships = {"noun": {"verb", "adj", "pronoun"},  "verb":{"noun", "adverb", "pronoun"}, "adj":{"noun", "pronoun"}, "pronoun":{"verb", "adj", "adverb", "noun"}, "adverb": {"verb"}}

    mask = np.ones((len(list_txt), len(list_txt))) #including the [CLS] and [SEP] tags 
    #relationships that we want noun->adj, adj->noun, verb->noun, noun->verb, adverb->verb, verb->adverb, pronouns->nouns, nouns->pronouns
    mask_indx = 6 #adjusted for [CLS] and [SEP] tokens 
    
    for row in range(mask.shape[0]):
        row_word_pos = word_pos[row] 
        if row_word_pos: #the [CLS], [SEP], and [MASK] tokens will be allowed to be influenced by every word 
            allowed_pos = relationships[row_word_pos] 
            for col in range(mask.shape[1]): 
                #for now, don't worry about the [mask] token not being allowed to influence the words 
                if word_pos[col] not in allowed_pos: 
                    mask[row,col] = 0
                # else:
                #     mask[row, col] = 0 
    np.fill_diagonal(mask, 1) #allow words to attend to themselves

    return mask 

In [68]:
txt = "[CLS] loony [mask] have ventured at least twice [mask] the future the [mask] time was tung the brilliantly funny [mask] [UNK] the latter time [mask] with this  um  effort [UNK] [UNK] isnt without merit and [mask] be considered a good product style\" it not that it isnt [mask] [mask] warner brothers quality wb cartoons whichever noted for their cheeky humor appealing at least as [mask] to [mask] as to lavished these pedestrian superhero episodes on the other hand [mask] fail to convince adults to pass them up the premise of the series is interpol [mask] ordinary individuals 2 bunnies a [UNK] devil a duck a roadrunner and a coyote live [mask] the [UNK] of [UNK] and [mask] super powers when a [mask] strikes the planet in [UNK] whats confusing is that the titles section features these individuals with a [UNK] to [UNK] from the 21st century cute but [UNK] stupid in each episode the super [UNK]  amid mildly amusing but essentially [mask] banter [mask] fight various super villains for the [mask] part these are types that appear in every mediocre superhero adventure series and even some of the better ones like many mediocre superhero series this one takes its villains far too seriously for the living and of course these guys [mask] the only characters that laugh  the usual evil laugh of course why [mask] it that villains in predictable superhero adventures [mask]  [mask]  laugh [UNK] at every [mask] animated [mask] [mask] this sort seems [mask] leave laughter exclusively in the [SEP]"
# mask = generate_mask(txt)
# import re 
l = txt.split(" ") 
print(l)
# with np.printoptions(threshold=np.inf):
#     print(mask)
# print(mask)

['[CLS]', 'loony', '[mask]', 'have', 'ventured', 'at', 'least', 'twice', '[mask]', 'the', 'future', 'the', '[mask]', 'time', 'was', 'tung', 'the', 'brilliantly', 'funny', '[mask]', '[UNK]', 'the', 'latter', 'time', '[mask]', 'with', 'this', '\x85', 'um', '\x85', 'effort', '[UNK]', '[UNK]', 'isnt', 'without', 'merit', 'and', '[mask]', 'be', 'considered', 'a', 'good', 'product', 'style"', 'it', 'not', 'that', 'it', 'isnt', '[mask]', '[mask]', 'warner', 'brothers', 'quality', 'wb', 'cartoons', 'whichever', 'noted', 'for', 'their', 'cheeky', 'humor', 'appealing', 'at', 'least', 'as', '[mask]', 'to', '[mask]', 'as', 'to', 'lavished', 'these', 'pedestrian', 'superhero', 'episodes', 'on', 'the', 'other', 'hand', '[mask]', 'fail', 'to', 'convince', 'adults', 'to', 'pass', 'them', 'up', 'the', 'premise', 'of', 'the', 'series', 'is', 'interpol', '[mask]', 'ordinary', 'individuals', '2', 'bunnies', 'a', '[UNK]', 'devil', 'a', 'duck', 'a', 'roadrunner', 'and', 'a', 'coyote', 'live', '[mask]', 'the

In [25]:
txt = "[CLS] i have seen the film a few days back on a video tape and even though it was hard [mask] swallow it [mask] one take because of its length and story i liked it very much i was impressed first by the script and [mask] by the realization of [mask] script the film takes you on a ride but [mask] is not an easy joyful ride [mask] goes [mask] time and different [mask] regimes and shows the influence of them to ordinary peoples lives [mask] i loved was the inner logic the film followed striking which just like logic in life was rather illogical and confusing at times [mask] [mask] the end when [mask] thought about it all the events and twists made sense it makes no sense though to try to [UNK] the story as it spreads in more than 50 years of time i also liked very much [UNK] [UNK] character [UNK] and the way mcewan played alluded as some critics would saw with restless abandon what i didnt like [mask] [mask] was that rehabilitation think he later played characters that [mask] me of [UNK] in films like [UNK] romance\" [UNK] [mask] which i actually love and to some [mask] in \"the insulted and the [UNK] [UNK] i [UNK] [UNK] shows [mask] think what a great filmmaker andrei [UNK] was before [mask] went to hollywood and made forgettable films like [UNK] and [UNK] and less [mask] like [UNK] train\" i would prefer [UNK] [UNK] to them [SEP]" 
test_txt = txt.replace("\", "")
print(test_txt)

SyntaxError: unterminated string literal (detected at line 2) (1906263203.py, line 2)

In [62]:
txt = "this is a \x100 test"
txt.split(" ")

['this', 'is', 'a', '\x100', 'test']

In [88]:
import keras
import tensorflow as tf
from keras import layers
from keras.layers import TextVectorization

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape("!#$%&'()*+,-./:;<=>?@\^_`{|}~"), ""
    )

#Testing text vectorization on custom spaces (ex: \x95) 
all_data = ['As predictable as a Hallmark card, but not without merit, The Rookie makes for a solid outing. Dennis Quaid, the most reasonable jock actor working today, is absolutely perfect as the science teacher turned baseball player Jimmy Morris. The film is never dumbed down for the children, as would be expected from a G rated film. As a sports film, The Rookie is one of the best I have seen since Any Given Sunday.',
 "If The Man in the White Suit had been done in America, can't you see either Danny Kaye or Jerry Lewis trying on Alec Guinness's Sidney Stratton on for size?<br /><br />This is one of the best of Alec Guinness's films and certainly one of the best that Ealing Studios in the United Kingdom ever turned out. It's so perfectly fits within the time frame of the new Labour government and the society it was trying to build. It's amazing how in times of crisis capital and labor can agree.<br /><br />Alec Guinness this meek little schnook of a man is obsessed with the idea that he can invent clothing that will never need cleaning, that in fact repels all kinds of foreign matter the minute it touches the garment. <br /><br />He's a persistent cuss and he does succeed. Of course the implications haven't really been thought through about the kind of impact clothing like that will have on society. In the end everyone is chasing him down like they would a fugitive, almost like Peter Lorre from M or Orson Welles in The Stranger or even Robert Newton in Oliver Twist. <br /><br />It's the mark of a great comedy film that a potentially serious situation like that chase as described in some of the serious films I've mentioned can be played for laughs. Poor Guinness's suit is not only white and stain repellent, but it glows like a neon sign.<br /><br />Other than Guinness the best performances are from Cecil Parker as yet another pompous oaf, Joan Greenwood as his siren daughter and Ernest Thesiger the biggest clothing manufacturer in the UK> <br /><br />Come to think of it, did Paramount borrow that suit from Ealing and give it to John Travolta for Saturday Night Fever?",
 "I went to see this movie at our college theater thirty years ago because I liked Bruce Dern in Silent Running and Family Plot. To this day (sorry Jack Nicholson), it is still the dullest movie I've ever seen. It just went on and on with no discernible point and then - it just ended. The lights came up and I watched everyone looking around in confusion. Had the projectionist missed a reel? I've never had the urge to find out. All I remember about the movie is that it was a non-drama about some annoying college basketball players and their coach. The most enjoyable part of the movie was watching the totally mystified audience afterwords. Fortunately, this was just an exception for Jack, Bruce, and Karen Black."]

def get_vectorize_layer(texts, vocab_size, max_seq, special_tokens=["[MASK]"]):
    """Build Text vectorization layer

    Args:
      texts (list): List of string i.e input texts
      vocab_size (int): vocab size
      max_seq (int): Maximum sequence lenght.
      special_tokens (list, optional): List of special tokens. Defaults to ['[MASK]'].

    Returns:
        layers.Layer: Return TextVectorization Keras Layer
    """
    vectorize_layer = TextVectorization(
        max_tokens=vocab_size,
        output_mode="int",
        standardize=custom_standardization,
        output_sequence_length=max_seq,
    )
    vectorize_layer.adapt(texts)

    # Insert mask token in vocabulary
    vocab = vectorize_layer.get_vocabulary()
    vocab = vocab[2 : vocab_size - len(special_tokens)] + ["[mask]"]
    vectorize_layer.set_vocabulary(vocab)
    return vectorize_layer


vectorize_layer = get_vectorize_layer(
    all_data,
    30000,
    256,
    special_tokens=["[mask]"],
)

id2token = dict(enumerate(vectorize_layer.get_vocabulary()))

def decode(tokens):
    return " ".join([id2token[t] for t in tokens if t != 0])




In [103]:
# vectorize_layer(["this is a move"])[0]
# test = "[CLS] castle in the sky is undoubtedly a hayao miyazaki film after seeing it [mask] the first time im glad to say that [mask] doesnt [mask] on the contrary you get your times worth which means as to what [mask] films are [mask] that is nothing less than excellent produced early in his [mask] career castle in the sky anticipates many of the trade marks in his later movies [mask] strong but vote [mask] characters forced to grow [mask] due to external circumstances helped out by very interesting and [mask] times lovable supporting [mask] and of course the usual battle of nature versus civilization [mask] [mask] lots of it [mask] painted sceneries  but alas no pigs at least that [mask] noticed after [mask] i have only seen chronic once never the less miyazaki had already [mask] his theatrical debut [mask] [mask] earlier [mask] [UNK] which was a dress rehearsal for princess mononoke his magnum opus castle in the [mask] is set a bit a part from [mask] two with a soft action packed first 30 minutes resembling tactics tv series conan and his directed episodes of [UNK] holmes in here we are introduced to [mask] a girl who literally falls from the sky only to be found by pazu a [mask] boy working in a little [mask] mining town intrigued by her amnesia and suspecting a connection between her and the mysterious flying city of laputa pazu is set on helping her find out [mask] [mask] [mask] from whilst escaping the army and a gang of [SEP]"
test = "castle in the sky" 
decode(vectorize_layer([test])[0].numpy())

'[UNK] in the [UNK]'

In [106]:
test = '\x96'
print(ord(test))

150


In [146]:
def generate_mask(vectorized_txt): 
    pos_vals = {"D", "J", "N", "P", "V", "R"}
    map_to_pos = {"D":"verb", "J":"adj", "N":"noun", "P":"pronoun", "V":"verb", "R": "adverb"}
    relationships = {"noun": {"verb", "adj", "pronoun"},  "verb":{"noun", "adverb", "pronoun"}, "adj":{"noun", "pronoun"}, "pronoun":{"verb", "adj", "adverb", "noun"}, "adverb": {"verb"}}

    
    decoded_txt = "" 
    word_count =  0 
    #assuming the shape of vectorized_txt is (256,) 
    for i in range(len(vectorized_txt)):
        vectorized_word = vectorized_txt[i] 
        if vectorized_word == 0:
            break 
        else:
            #doing this in case the last word is a specialized space token and it gets stripped at the end 
            if i != len(vectorized_txt)-1 and vectorized_txt[i+1] != 0 :
                decoded_txt += decode([vectorized_word]) + " " 
            else:
                decoded_txt += decode([vectorized_word]) 
            word_count += 1 
    
    tokenized = word_tokenize(decoded_txt) 
    tagged = nltk.pos_tag(tokenized)
    word_pos = []
    for word, pos in tagged: 
        if word == ']':
            word_pos.pop(-1)
            word_pos.pop(-1)
            word_pos.append(None)
        if ord(word[0]) >= 127 and len(word) == 1 or pos[0] not in pos_vals: 
            word_pos.append(None)
        else:
            word_pos.append(map_to_pos[pos[0]]) 

    mask = np.ones((len(word_pos), len(word_pos)))

    for row in range(mask.shape[0]):
        row_word_pos = word_pos[row] 
        if row_word_pos: #the [CLS], [SEP], and [MASK] tokens will be allowed to be influenced by every word 
            allowed_pos = relationships[row_word_pos] 
            for col in range(mask.shape[1]): 
                #for now, don't worry about the [mask] token not being allowed to influence the words 
                if word_pos[col] not in allowed_pos: 
                    mask[row,col] = 0
                # else:
                #     mask[row, col] = 0 

    np.fill_diagonal(mask, 1) 

    return mask 
                
    # for vectorized_word in vectorized_txt: 
    #     if vectorized_word == 0:
    #         break 
    #     else:
    #         decoded_txt += decode(vectorized_word) + " "
    #         word_count += 1 
    # decoded_txt = decoded_txt.strip() 

    # tokenized = sent_tokenize(decoded_txt) 


    # mask = np.ones((len(tagged) + 2, len(tagged) + 2)) 
    


    


In [119]:
temp = "this is a \x86 test" 
tokenized = word_tokenize(temp) 
tagged = nltk.pos_tag(tokenized)



[('this', 'DT'), ('is', 'VBZ'), ('a', 'DT'), ('\x86', 'JJ'), ('test', 'NN')]


In [141]:
txt = "\x86"
print(txt)
print(len(txt))
print(ord('\x80'))


1
128


In [143]:
test = [    1,  8932,  3655, 29999, 21983,    30,   218,  1442, 29999,
           2,   715,     2,    88,    58,    13,    15,     2,  2108,
       29999,  4900,     1,     2,  1591,    58,    13,    15,    11,
        5451,  7350,  5451,   759,     1,  8723, 29999,   200,  3626,
           3,   227,    26,  1183, 29999,    49,  2138,    66,     9,
          21,    12,     9,   202,    55, 29999,  2932,  1018,   484,
       10145, 29999,    23,  3061,    16,    63, 13880,   454,  2411,
          30, 29999,    14,    73,     6, 29999,    14,     6,   460,
         129,  6850,  4437,   680,    20,     2,    76,   565,   560,
        1864, 29999,  2356,  1508,     6,  1312,    90,    55,     2,
         831,     5,     2,   209, 29999,    12,  1350,  1800,  3183,
         283, 19353, 29999,     1, 29999,     4, 29999,     4, 29999,
           3,     4,  9072,   433,    20,     2,     1,     5,     1,
           3, 11201,  1576,  1764,    50,     4,  7267,  3620,     2,
        1278,     8,     1,   691,  1417,     7,    12,     2,  2858,
        2510,   889,   129,  3183,    15,     4,     1,     6,     1,
          35,     2,  5900,  1105,  1020,    18,     1,   359,     8,
         246,   389,     2, 29999,     1,   488,  8447, 29999,  1117,
          18,  1904,  5351,  6941,   488,   547,   945,  1576, 29999,
       29999,     2,    87,   170,   129,    23,  2123,    12,   952,
           8, 29999,  1527, 29999,  1228,   209,     3,    54,    46,
           5,     2,   123,   512,    38,   104, 29999,  4437,   209,
          11,    28,   301, 29999,  1765,   226, 29999,   607, 29999,
           2,  1986,     3,     5,   258,   129,   407,    23,     2,
          60, 29999,    12,   434,   488,     2,   648,   468,   434,
           5,   258, 29999,     7,     9,    12,  1765, 29999,   706,
       29999,  2454,   205,   488,   205,   488, 29999,     1,    30,
         166,  1396,  1258,   814,     5, 29999,   421,   180, 29999,
         549,  2109,  6974,     8]

In [147]:
mask = generate_mask(test)
mask

KeyError: 8932